# Advent of Code - 2024 - Day 16 - Problem 2

https://adventofcode.com/2024/day/16

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day16.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

# DATA = """#################
# #...#...#...#..E#
# #.#.#.#.#.#.#.#.#
# #.#.#.#...#...#.#
# #.#.#.#.###.#.#.#
# #...#.#.#.....#.#
# #.#.#.#.#.#####.#
# #.#...#.#.#.....#
# #.#.#####.#.###.#
# #.#.#.......#...#
# #.#.###.#####.###
# #.#.#...#.....#.#
# #.#.#.#####.###.#
# #.#.#.........#.#
# #.#.#.#########.#
# #S#.............#
# #################"""
# # DATA = """####
# # #..E#
# # #...#
# # #S..#
# # #####"""
# DATA = list(map(str.strip, DATA.splitlines()))

# DATA

## Create Path Class

In [2]:
ORIENTATION_NORTH = (-1, 0)
ORIENTATION_SOUTH = (1, 0)
ORIENTATION_WEST = (0, -1)
ORIENTATION_EAST = (0, 1)
ORIENTATIONS = [ORIENTATION_NORTH, ORIENTATION_SOUTH, ORIENTATION_WEST, ORIENTATION_EAST]


class Path:

    def __init__(self, location, orientation):
        self._locations = [location]
        self._orientations = [orientation]
        self._costs = [0]

        self._location_set = set()
        self._location_set.add(location)

    @staticmethod
    def _add(location, orientation):
        return (location[0] + orientation[0], location[1] + orientation[1])

    def extend(self, location, orientation, cost):
        self._locations.append(location)
        self._orientations.append(orientation)
        self._costs.append(cost)

        self._location_set.add(location)

    def get_possible_steps(self):
        location = self._locations[-1]
        orientation = self._orientations[-1]
        cost = self._costs[-1]

        if orientation == ORIENTATION_NORTH:
            yield (Path._add(location, ORIENTATION_NORTH), ORIENTATION_NORTH, cost + 1)
            yield (Path._add(location, ORIENTATION_WEST), ORIENTATION_WEST, cost + 1001)
            yield (Path._add(location, ORIENTATION_EAST), ORIENTATION_EAST, cost + 1001)
        elif orientation == ORIENTATION_EAST:
            yield (Path._add(location, ORIENTATION_EAST), ORIENTATION_EAST, cost + 1)
            yield (Path._add(location, ORIENTATION_NORTH), ORIENTATION_NORTH, cost + 1001)
            yield (Path._add(location, ORIENTATION_SOUTH), ORIENTATION_SOUTH, cost + 1001)
        elif orientation == ORIENTATION_SOUTH:
            yield (Path._add(location, ORIENTATION_SOUTH), ORIENTATION_SOUTH, cost + 1)
            yield (Path._add(location, ORIENTATION_EAST), ORIENTATION_EAST, cost + 1001)
            yield (Path._add(location, ORIENTATION_WEST), ORIENTATION_WEST, cost + 1001)
        else:  # ORIENTATION_WEST
            yield (Path._add(location, ORIENTATION_WEST), ORIENTATION_WEST, cost + 1)
            yield (Path._add(location, ORIENTATION_SOUTH), ORIENTATION_SOUTH, cost + 1001)
            yield (Path._add(location, ORIENTATION_NORTH), ORIENTATION_NORTH, cost + 1001)

    def contains_location(self, location):
        return location in self._location_set

    def get_cost(self):
        return self._costs[-1]

    def check_point(self):
        return len(self._locations)

    def backtrack(self, memento):
        for idx in range(len(self._locations) - memento):
            location = self._locations.pop()
            self._orientations.pop()
            self._costs.pop()

            self._location_set.remove(location)

## Create Map Class

In [3]:
import sys

CHAR_START = "S"
CHAR_END = "E"
CHAR_WALL = "#"
CHAR_SPACE = "."


class Map:

    def __init__(self, lines):

        self._lines = list()
        row = -1
        for line in lines:
            row += 1
            chars = list()
            self._lines.append(chars)
            col = -1
            for char in line:
                col += 1
                chars.append(char)

                if char == CHAR_START:
                    self._start = (row, col)
                elif char == CHAR_END:
                    self._end = (row, col)

        self._location_costs = dict()

    def _is_empty_or_end(self, location):
        row, col = location
        return self._lines[row][col] in [CHAR_SPACE, CHAR_END]

    def _is_end(self, location):
        row, col = location
        return self._lines[row][col] == CHAR_END

    def _is_better_location_cost(self, location, cost):
        if location in self._location_costs:
            if cost > self._location_costs[location] + 5000:
                return False
            if cost >= self._location_costs[location]:
                return True

        self._location_costs[location] = cost
        return True

    def find_best_locations(self):
        path = Path(self._start, ORIENTATION_EAST)
        max_cost = 98520
        best_cost = max_cost
        best_locations = set()
        for new_max_cost in self.find_solutions(path, max_cost):
            max_cost = new_max_cost
            if max_cost < best_cost:
                # self._location_costs = dict()
                best_cost = max_cost
                best_locations = set()
            best_locations.update(path._location_set)
        return best_locations

    def find_solutions(self, path, max_cost):
        memento = path.check_point()
        for next_location, next_orientation, next_cost in path.get_possible_steps():
            path.backtrack(memento)
            if not self._is_better_location_cost(next_location, next_cost):
                pass
            elif not self._is_empty_or_end(next_location):
                # Don't step into a wall!
                pass
            elif path.contains_location(next_location):
                # Don't step back into a location we've already visited
                pass
            elif next_cost > max_cost:
                # This isn't a better solution than to one we've already found. Ignore it.
                pass
            else:  # Potential solution
                path.extend(next_location, next_orientation, next_cost)
                if self._is_end(next_location):
                    # Establish a new max_cost. Used for finding alternative solutions below:
                    max_cost = next_cost
                    yield max_cost
                else:
                    for other_cost in self.find_solutions(path, max_cost):
                        max_cost = other_cost
                        yield max_cost

## Parse Data

Parses the input `DATA` into `MAP`.

In [4]:
MAP = Map(DATA)

best_locations = MAP.find_best_locations()

print(f"best_locations = {len(best_locations)}")

best_locations = 609
